In [ ]:
import pickle
import numpy as np
import pandas as pd
import gc
import random
from custom_metric import *

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

with open('D:/inputs/24/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)
with open('D:/inputs/24/test.pickle', 'rb') as f:
    test = pickle.load(f)
    
    
LEN_TRAIN = train.shape[0]

In [ ]:
train = np.delete(train, [0,1,2,3,4,5,6], axis = 0) # 200x200으로 맞추기 위해서. 7개 날림.
train = train.reshape(75950 // (5*5), 200, 200, 14) 

train_y = np.delete(train_y, [0,1,2,3,4,5,6], axis = 0)
train_y = train_y.reshape(75950 // (5*5), 200, 200, 1)

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras import Model
from tensorflow.keras import layers

In [ ]:
def build_model(input_layer):
    
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, padding='same', name='block1_conv1')(input_layer)
    x = layers.BatchNormalization(name='block1_conv1_bn')(x)
    x = layers.Activation('relu', name='block1_conv1_act')(x)
    x = layers.Conv2D(64, (3, 3),  padding='same', use_bias=False, name='block1_conv2')(x)
    x = layers.BatchNormalization(name='block1_conv2_bn')(x)
    x = layers.Activation('relu', name='block1_conv2_act')(x)

    residual = layers.Conv2D(128, (1, 1),strides=(2, 2),padding='same',use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.SeparableConv2D(128, (3, 3),padding='same',use_bias=False,name='block2_sepconv1')(x)
    x = layers.BatchNormalization(name='block2_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block2_sepconv2_act')(x)
    x = layers.SeparableConv2D(128, (3, 3),padding='same',use_bias=False,name='block2_sepconv2')(x)
    x = layers.BatchNormalization(name='block2_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3),strides=(2, 2), padding='same',name='block2_pool')(x)
    x = layers.add([x, residual])


    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False,name=prefix + '_sepconv1')(x)
        x = layers.BatchNormalization(name=prefix + '_sepconv1_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = layers.SeparableConv2D(128, (3, 3), padding='same',use_bias=False,name=prefix + '_sepconv2')(x)
        x = layers.BatchNormalization(name=prefix + '_sepconv2_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = layers.SeparableConv2D(128, (3, 3), padding='same',use_bias=False,name=prefix + '_sepconv3')(x)
        x = layers.BatchNormalization(name=prefix + '_sepconv3_bn')(x)
        x = layers.add([x, residual])

    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu', name='block13_sepconv1_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),padding='same',use_bias=False,name='block13_sepconv1')(x)
    x = layers.BatchNormalization(name='block13_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block13_sepconv2_act')(x)
    x = layers.SeparableConv2D(1024, (3, 3),padding='same',use_bias=False,name='block13_sepconv2')(x)
    x = layers.BatchNormalization(name='block13_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3),strides=(2, 2),padding='same',name='block13_pool')(x)
    x = layers.add([x, residual])

    x = layers.SeparableConv2D(1536, (3, 3),padding='same',use_bias=False,name='block14_sepconv1')(x)
    x = layers.BatchNormalization(name='block14_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block14_sepconv1_act')(x)

    x = layers.SeparableConv2D(2048, (3, 3),padding='same',use_bias=False,name='block14_sepconv2')(x)
    x = layers.BatchNormalization(name='block14_sepconv2_bn')(x)
    x = layers.Activation('relu', name='block14_sepconv2_act')(x)
    x = Conv2D(1, (1,1), padding="same", activation='relu')(x)
    x = Conv2DTranspose(1, (3, 3), strides=(2, 2), padding="same")(x)
    x = Conv2DTranspose(1, (3, 3), strides=(2, 2), padding="same")(x)
    x = Conv2DTranspose(1, (3, 3), strides=(2, 2), padding="same")(x)

    return x
    
input_layer = Input((200, 200, 14))
output_layer = build_model(input_layer)
model = Model(input_layer, output_layer)

# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
model.compile(loss="mae", optimizer="adam", metrics=["mae"])

scores = []
model.summary()

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model

def build_model(input_layer, start_neurons):
    
    # 40 x 40 -> 20 x 20
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.25)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.25)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    return output_layer

input_layer = Input((40, 40, train.shape[3]))
output_layer = build_model(input_layer, 32)
model = Model(input_layer, output_layer)

# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
model.compile(loss="mae", optimizer="adam", metrics=["mae"])

scores = []



In [ ]:
model.summary()

In [ ]:
# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras]) # custom metric으로 하면 실행이 안됨
model.compile(loss="mae", optimizer="adam", metrics=['mae'])

model.summary()

model.fit(train, train_y, batch_size = 100, epochs = 1)

# model.predict(test2)